## Train apt/dpk test valdist cbk

In [ ]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import glob
import importlib

import deepposekit as dpk

import TrainingGeneratorTFRecord as TGTFR
import apt_dpk 
import run_apt_expts_2 as rae
import APT_interface as apt
import PoseTools as pt
import multiResData as mrd
import tfdatagen
import apt_dpk_exps as ade

import time
from os.path import expanduser

import mpl_toolkits.axes_grid1 as axg1


In [ ]:
expname = '/dat0/apt/cache/leap_dset/dpkfly/view_0/dpkorig_20200512T180752_run4_tfr_ptia_01_TEST'

In [ ]:
ade.exp2orig_train(expname, shortdebugrun=True)

In [ ]:
picf='/dat0/apt/cache/leap_dset/dpkfly/view_0/dpkorig_20200512T180752_run4_tfr_ptia_01_TEST/trn20200614T101737.vdist.pickle'

In [ ]:
pic=pt.pickle_load(picf)

In [ ]:
pic.keys()


In [ ]:
for k,v in pic.items():
    print("{}: {}".format(k, len(v)))

In [ ]:
import csv

In [ ]:
with open('/dat0/apt/dpkwking/dumba.foo.log','a') as f:
    writer = csv.writer(f)
    writer.writerow(['asd'])

In [ ]:
pwd

In [ ]:
# Create a TGTFR with our default imgaug
LEAPSTRIPPEDLBL = '/dat0/jrcmirror/groups/branson/bransonlab/apt/experiments/data/leap_dataset_gt_stripped.lbl'
EXPNAME = 'val10pct'
CACHE = '/dat0/apt/cache'
RNGSEED = 17

dg = dpk.DataGenerator(DPK_DSET)

conf = apt.create_conf(LEAPSTRIPPEDLBL, 0, EXPNAME, \
                       CACHE, 'dpkfly', quiet=False)
conf.img_dim = 1  # hack, the leap stripped lbl has NumChans=3, but we created the tfr 
                  # directly using the dpk h5 which is 1-channel
conf = apt_dpk.update_conf_dpk(conf,
                               dg.graph,
                               dg.swap_index,
                               n_keypoints=dg.n_keypoints,
                               imshape=dg.compute_image_shape(),
                               useimgaug=True,
                               imgaugtype='dpkfly')
tgtfr = TGTFR.TrainingGeneratorTFRecord(conf)


In [ ]:
tgtfr.conf.dpk_augmenter.reseed(RNGSEED)
tgtfr.use_tfdata = False
gtf = tgtfr(n_outputs=1, batch_size=8, 
       validation=False, confidence=True, shuffle=False)

imstgts_apt = [next(gtf) for _ in range(3)]

imsapt, tgtsapt = tfdatagen.xylist2xyarr(imstgts_apt, xisscalarlist=True)
imsapt.shape, tgtsapt.shape

In [ ]:
tgtfr.conf.dpk_augmenter.reseed(RNGSEED)
tgtfr.use_tfdata = True

In [ ]:
ds = tgtfr(n_outputs=2, batch_size=8, 
       validation=False, confidence=True, shuffle=False)

imstgts_ds = tfdatagen.read_ds_idxed(ds, range(3))

imsDS, tgtsDS = tfdatagen.xylist2xyarr(imstgts_ds)
imsDS.shape, tgtsDS.shape

In [ ]:
np.allclose(imsDS,imsapt), np.allclose(tgtsDS,tgtsapt), \
np.array_equal(imsDS,imsapt), np.array_equal(tgtsDS,tgtsapt), \

In [ ]:
'''
dg = dpk.DataGenerator(DPK_DSET)
ia = apt_dpk.make_imgaug_augmenter('dpkfly', dg)
DSFAC = 2
SIGMA = 5
VALSPLIT = 0.0
GRAPHSCALE = 1
tg = dpk.TrainingGenerator(generator=dg,
                           downsample_factor=DSFAC,
                           augmenter=ia,
                           use_graph=True,
                           shuffle=True,
                           sigma=SIGMA,
                           validation_split=VALSPLIT,
                           graph_scale=GRAPHSCALE,
                           random_seed=0)
'''

In [ ]:
tgtfr.use_tfdata

In [ ]:
sdn = dpk.models.StackedDenseNet(tgtfr, 
                                 n_stacks=2, 
                                 growth_rate=48, 
                                 pretrained=True)


#### Callbacks

In [ ]:
conf.cachedir

In [ ]:
cbk_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                              factor=0.2, 
                                              verbose=1, 
                                              patience=20)

cbk_mdlcpt = tf.keras.callbacks.ModelCheckpoint(
    conf.cachedir + "/best_model_sdn.h5",
    monitor="val_loss",  # monitor="loss" # use if validation_split=0
    verbose=1,
    save_best_only=True,
)

#early_stop = EarlyStopping(
#    monitor="val_loss",
#    # monitor="loss" # use if validation_split=0
#    min_delta=0.001,
#    patience=100,
#    verbose=1
#)

callbacks = [cbk_lr, cbk_mdlcpt]

In [ ]:
?tf.keras.Model.fit_generator


In [ ]:
?tf.keras.Model.fit

In [ ]:
BSIZE = 8
DLSTEPS = 40000
STEPSPEREPOCH = 50
EPOCHS = DLSTEPS//STEPSPEREPOCH
VALSTEPS = 0  # we aren't using val for anything

In [ ]:
EPOCHS

In [ ]:
sdn.n_outputs

In [ ]:
dstrn = sdn.train_generator(sdn.n_outputs,8,
                            validation=False,
                            confidence=True,
                            shuffle=True,
                            infinite=True)
                            


In [ ]:
resTrn=tfdatagen.read_ds_idxed(dstrn,range(3))


In [ ]:
dsval = sdn.train_generator(sdn.n_outputs,8,
                            validation=True,
                            confidence=True,
                            shuffle=False,
                            infinite=False)

In [ ]:
'''
sdn.fit(
    batch_size=BSIZE, #    validation_batch_size=BSIZE,
    callbacks=callbacks,
    epochs=EPOCHS,
    steps_per_epoch=STEPSPEREPOCH, #    validation_steps=VALSTEPS,
)
'''

In [ ]:
sdn.activate_callbacks(callbacks)

In [ ]:
m=sdn.train_model

In [ ]:
m.compile('adam','mse')

In [ ]:
m.fit(dstrn,
      epochs=10,
      steps_per_epoch=5,
      verbose=1,
      callbacks=callbacks,
      validation_data=dsval,
      validation_steps=int(np.ceil(150/8)) )


In [ ]:
1350/8

#### Train with APT using dflt IA. 

Freeze all artifacts/inputs. Compare results

#### Train with APT using PT and Bub aug params.
Freeze all artifacts/inputs. Compare results.
(TODO: add this to data_pipeline nb)